In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'd:\\kidney-disease-classification'

In [3]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/Sidd-77/kidney-disease-classification.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="Sidd-77"
os.environ['MLFLOW_TRACKING_PASSWORD']="61bcab158d9ab43c4be35facef8400397b011fcc"

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model('artifacts/training/model.keras')


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvalutationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    

In [7]:
from CNN_Classifier import *
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self)->EvalutationConfig:
        eval_config = EvalutationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-data/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/Sidd-77/kidney-disease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    
    


In [9]:
import tensorflow as tf
import os
from pathlib import Path
import mlflow
import mlflow.keras 
from urllib.parse import urlparse

In [10]:
class Evaluation:
    def __init__(self, config: EvalutationConfig):
        self.config = config
        self.validation_ratio=0.2

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def _valid_generator(self):
        self.valid_generator = tf.keras.utils.image_dataset_from_directory(
                self.config.training_data,
                labels="inferred",
                label_mode="categorical",
                class_names=['Cyst', 'Normal', 'Stone', 'Tumor'],
                color_mode="rgb",
                batch_size=self.config.params_batch_size,
                image_size=self.config.params_image_size[:-1],
                shuffle=True,
                seed=69,
                validation_split=self.validation_ratio,
                subset="validation",
                interpolation="bilinear",
                verbose=True,
        )

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != 'file':
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [11]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evalutaion = Evaluation(eval_config)
    evalutaion.evaluation()
    evalutaion.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-05 12:15:50,518: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-04-05 12:15:50,520: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-04-05 12:15:50,521: INFO: common: created directory at: artifacts:]
Found 1200 files belonging to 4 classes.
Using 240 files for validation.
15/15 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.2210 - loss: 1.3904
[2024-04-05 12:16:19,710: INFO: common: json file saved at: scores.json:]


2024/04/05 12:16:22 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/04/05 12:16:54 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 4
Created version '4' of model 'VGG16Model'.
